Mass Spring Model
------------------

A mass sping model is a decent approximation for cloth or any thin shelled material.  The model is made up of a mesh of nodes interconnected with edges.  The nodes have a mass $m$ and the edges have a spring contant $k$, length $len$, and a natural resting length $len_{rest}$.  The force of gravity is included with, using the normal gravitational constant, $g=<0,0,-9.8>$.

Thus the force on each node is:
$$F = mg + \sum_{springs} k(len - len_{rest})$$

Then Newton's second equation provides the differential equations to update the position of the nodes, $x$.  A damepning factor $\lambda$ is included to approximate the effect of friction.  We use two coupled first order equations for each node:
$$\frac{dx}{dt} = v$$
$$\frac{dv}{dt} = \frac{F - \lambda v}{m}$$

Given this formulation of differential equations an initial velocity must be supplied, zero is the natural choice.

In order to solve the differential equations euler's method is used.

In [3]:
# Python IMPORTS
import numpy as np
import scipy as sp

In [4]:
# ODES   - Ordinary Differential Equations

# t - time [s]
# X = [pos_x, pos_y, pos_z, vel_x, vel_y, vel_z]
# forceVector - [force_x, force_y, force_z]
# 
def newtonEq(t, X, mass, forceVector, dampeningFactor, links, particle):
    X_prime = np.array(X.len);
    # position
    X_prime[0] = X[3];
    X_prime[1] = X[4];
    X_prime[2] = X[5];
    
    # velocity
    springforce = np.zeros(3);
    for link in links:
        springforce += link.getForce(particle);
    X_prime[3] = ((forceVector[0] + springforce[0]) - dampeningFactor*X[3])/mass;   # model adds velocity dampening in place
    X_prime[4] = ((forceVector[1] + springforce[1]) - dampeningFactor*X[4])/mass;   # of internal friction
    X_prime[5] = ((forceVector[2] + springforce[2]) - dampeningFactor*X[5])/mass;
    return X_prime;

def integrate(t, dt, X, dampeningFactor, particle):
    method = sp.integrate.ode(newtonEq).set_integrator('dopri5');  # runge-katta 4-5 method
    method.set_initial_value(X, t).set_f_params( particle.mass, particle.forceVector, dampeningFactor, particle.links, particle);
    return method.integrate(method.t+dt);


SyntaxError: invalid syntax (<ipython-input-4-ada8107ceeb6>, line 13)

In [5]:
# mesh class design influenced by https://github.com/valorcurse/cloth-simulation
#   used under WTFPL license - http://www.wtfpl.net/

class Particle:
    def __init__(self, posvel, mass):
        self.posvel = posvel
        self.lastPosvel = posvel
        self.force =  np.zeros(3);
        forceVector[2] -= self.mass * -9.81;     # gravity in z direction
        self.mass = mass
        self.anchored = False
        self.links = []
    
    def update(self, et, dt):
        if(not self.anchored):
            self.posvel = integrate(et, dt, self.posvel, self.mass, self.forceVector, dampeningFactor, self);
        
    def applyForce(self, vec3d):
        self.force += vec3d;
        
    def applyConstraints(self):
        for link in self.links:
            link.applyConstraints()
        
class Link:
    def __init__(self, part1, part2, restingDist, k):
        self.particle1 = part1
        self.particle2 = part2
        self.restingDistance = restingDist
        self.springConstant = k
        
    def getForce(self, particle):
        # direction is with respect to particle in the argument
        #   spring force is always directed along the length of the spring
        #   either towards or away from the center
        if(particle == self.particle1):
            part1 = self.particle1;
            part2 = self.particle2;
        elif(particle == self.particle2):
            part2 = self.particle1;
            part1 = self.particle2;
        else:
            raise ValueError('a bad thing happened, the specified particle to use as reference location is not in this Link')
    
        # displacement in 1D
        dz = part2.posvel[2] - part1.posvel[2];
        dy = part2.posvel[1] - part1.posvel[1];
        dx = part2.posvel[0] - part1.posvel[0];
        currLen = np.sqrt(dx * dx + dy * dy + dz * dz);
        # spring force
        mag_force = self.springConstant * (currLen - self.restingDistance);    # hooke's law F = kx
        
        # place force in the direction of the spring
        springForce =  np.zeros(3);
        springForce[0] = (dx/currLen) * mag_force;
        springForce[1] = (dy/currLen) * mag_force;
        springForce[2] = (dz/currLen) * mag_force;
        






In [ ]:
def generateShirtMesh():

class Shirt:
    def __init__(self, density, springConstant, controlPoints):
        self.mesh = generateShirtMesh()